# Create Pump.Fun Token And Bundle 5 Wallets.

In [1]:
!pip install requests solders

In [2]:
import requests
import json
import base58
from solders.keypair import Keypair
from solders.transaction import VersionedTransaction
from solders.rpc.config import RpcSendTransactionConfig
from solders.rpc.requests import SendVersionedTransaction
from solders.commitment_config import CommitmentLevel

# Constants
PUMP_PORTAL_API = "https://pumpportal.fun/api/trade-local"
JITO_BUNDLE_API = "Your Quicknode Jito Bundler Endpoint URL here"
RPC_ENDPOINT = "Your RPC Endpoint here - Eg: https://api.mainnet-beta.solana.com/"
TOKEN_METADATA = {
    'name': 'PPTest',
    'symbol': 'TEST',
    'description': 'This is an example token created via PumpPortal.fun',
    'twitter': 'https://x.com/a1lon9/status/1812970586420994083',
    'telegram': 'https://x.com/a1lon9/status/1812970586420994083',
    'website': 'https://pumpportal.fun',
    'showName': 'true'
}
IMAGE_PATH = './example.png'



In [3]:
# List of private keys for Jito Bundle
PRIVATE_KEYS = [
    "Wallet A base 58 private key here",
    "Wallet B base 58 private key here",
    "Wallet C base 58 private key here",
    "Wallet D base 58 private key here",
    "Wallet E base 58 private key here"

SyntaxError: incomplete input (3598979297.py, line 7)

In [4]:
class PumpPortalTrader:
    def __init__(self):
        self.signer_keypair = Keypair.from_base58_string('your-base58-private-key')
        self.mint_keypair = Keypair()

    def create_ipfs_metadata(self):
        with open(IMAGE_PATH, 'rb') as f:
            file_content = f.read()

        files = {'file': ('example.png', file_content, 'image/png')}
        metadata_response = requests.post("https://pump.fun/api/ipfs", data=TOKEN_METADATA, files=files)
        return metadata_response.json()['metadataUri']

    def create_token(self):
        metadata_uri = self.create_ipfs_metadata()
        token_metadata = {
            'name': TOKEN_METADATA['name'],
            'symbol': TOKEN_METADATA['symbol'],
            'uri': metadata_uri
        }

        response = requests.post(
            PUMP_PORTAL_API,
            headers={'Content-Type': 'application/json'},
            data=json.dumps({
                'publicKey': str(self.signer_keypair.pubkey()),
                'action': 'create',
                'tokenMetadata': token_metadata,
                'mint': str(self.mint_keypair.pubkey()),
                'denominatedInSol': 'true',
                'amount': 1,  # Dev buy of 1 SOL
                'slippage': 10,
                'priorityFee': 0.0005,
                'pool': 'pump'
            })
        )

        tx = VersionedTransaction(VersionedTransaction.from_bytes(response.content).message, [self.mint_keypair, self.signer_keypair])
        
        commitment = CommitmentLevel.Confirmed
        config = RpcSendTransactionConfig(preflight_commitment=commitment)
        txPayload = SendVersionedTransaction(tx, config)

        response = requests.post(
            RPC_ENDPOINT,
            headers={"Content-Type": "application/json"},
            data=SendVersionedTransaction(tx, config).to_json()
        )
        txSignature = response.json()['result']
        print(f'Transaction: https://solscan.io/tx/{txSignature}')

    def send_create_tx_bundle(self):
        signerKeypairs = [Keypair.from_base58_string(key) for key in PRIVATE_KEYS]
        metadata_uri = self.create_ipfs_metadata()
        token_metadata = {
            'name': TOKEN_METADATA['name'],
            'symbol': TOKEN_METADATA['symbol'],
            'uri': metadata_uri
        }

        bundledTransactionArgs = [
            {
                'publicKey': str(signerKeypairs[0].pubkey()),
                'action': 'create',
                'tokenMetadata': token_metadata,
                'mint': str(self.mint_keypair.pubkey()),
                'denominatedInSol': 'false',
                'amount': 1000000,  # Dev buy of 1000000 tokens
                'slippage': 10,
                'priorityFee': 0.0005,
                'pool': 'pump'
            },
            {
                "publicKey": str(signerKeypairs[1].pubkey()),
                "action": "buy",
                "mint": str(self.mint_keypair.pubkey()),
                "denominatedInSol": "false",
                "amount": 1000000,
                "slippage": 50,
                "priorityFee": 0.0001,
                "pool": "pump"
            },
            {
                "publicKey": str(signerKeypairs[2].pubkey()),
                "action": "buy",
                "mint": str(self.mint_keypair.pubkey()),
                "denominatedInSol": "false",
                "amount": 1000000,
                "slippage": 50,
                "priorityFee": 0.0001,
                "pool": "pump"
            },
            {
                "publicKey": str(signerKeypairs[3].pubkey()),
                "action": "buy",
                "mint": str(self.mint_keypair.pubkey()),
                "denominatedInSol": "false",
                "amount": 1000000,
                "slippage": 50,
                "priorityFee": 0.0001,
                "pool": "pump"
            },
            {
                "publicKey": str(signerKeypairs[4].pubkey()),
                "action": "buy",
                "mint": str(self.mint_keypair.pubkey()),
                "denominatedInSol": "false",
                "amount": 1000000,
                "slippage": 50,
                "priorityFee": 0.0001,
                "pool": "pump"
            }
        ]

        response = requests.post(
            PUMP_PORTAL_API,
            headers={"Content-Type": "application/json"},
            json=bundledTransactionArgs
        )

        if response.status_code != 200:
            print("Failed to generate transactions.")
            print(response.reason)
        else:
            encodedTransactions = response.json()
            encodedSignedTransactions = []
            txSignatures = []

            for index, encodedTransaction in enumerate(encodedTransactions):
                if bundledTransactionArgs[index]["action"] == "create":
                    signedTx = VersionedTransaction(VersionedTransaction.from_bytes(base58.b58decode(encodedTransaction)).message, [self.mint_keypair, signerKeypairs[index]])
                else:
                    signedTx = VersionedTransaction(VersionedTransaction.from_bytes(base58.b58decode(encodedTransaction)).message, [signerKeypairs[index]])
                
                encodedSignedTransactions.append(base58.b58encode(bytes(signedTx)).decode())
                txSignatures.append(str(signedTx.signatures[0]))

            jito_response = requests.post(
                JITO_BUNDLE_API,
                headers={"Content-Type": "application/json"},
                json={
                    "jsonrpc": "2.0",
                    "id": 1,
                    "method": "sendBundle",
                    "Oops, something broke. Talk to me later?